In [30]:
from tqdm import tqdm
import torch
import os
import sys
sys.path.append("/scratch/m000122/bcabrown/continual-learning")
sys.path.append("/scratch/m000122/bcabrown/continual-learning/third_party/cartridges")
os.environ["CARTRIDGES_DIR"] = "/scratch/m000122/bcabrown/continual-learning/third_party/cartridges"
os.environ["CARTRIDGES_OUTPUT_DIR"] = "/scratch/m000122/bcabrown/continual-learning/third_party/cartridges/output"
from third_party.cartridges.cartridges.datasets import read_conversations
from third_party.cartridges.cartridges.data.finer.evals import LLAMA_CARTRIDGE_TEMPLATE
import torch.nn.functional as F
import math

In [2]:
data_path = "/scratch/m000122/stalaei/logs/continual_learning/data/finer_v1_train_ICL_exclude_current_250_triplets_false_1000_reps_temp_0.7/dataset.jsonl"
data = read_conversations(data_path)

In [55]:
row = data[2]

In [56]:
num_output_ids = len(row["output_ids"])
for idx in range(num_output_ids):
    topk_token_ids = row["topk_token_ids"][idx]
    topk_token_probs = row["topk_logprobs"][idx]
    output_id = row["output_ids"][idx]
    try:
        rank = topk_token_ids.index(output_id)
        print(output_id, rank, math.exp(topk_token_probs[rank]))
    except ValueError:
        idx = -1
        print(output_id, "not in topk", len(topk_token_ids))
    

1271 0 1.0
8417 0 1.0
279 0 1.0
8475 0 1.0
75760 0 0.9999998807907033
4833 0 0.9999998807907033
2326 0 1.0
27438 0 1.0
2599 0 1.0
4877 0 1.0
369 0 1.0
10768 0 0.9999997615814209
5120 0 1.0
13 0 1.0
21 0 1.0
19727 0 0.9999988079071329
304 0 1.0
279 0 1.0
2728 0 1.0
11914 0 1.0
11 0 1.0
584 0 1.0
1205 0 1.0
311 0 1.0
10765 0 1.0
279 0 1.0
7434 0 1.0
1694 0 1.0
7633 0 1.0
13 0 1.0
763 0 1.0
420 0 1.0
1162 0 1.0
11 0 1.0
279 0 1.0
7434 0 1.0
374 0 1.0
279 0 1.0
7782 1 0.2758625551992088
3430 0 0.9904612900127554
315 0 0.9729535584321016
264 0 0.9110561008937156
2626 0 0.9999974966048697
10824 0 0.9943897129536902
382 0 0.7772521936652804
59 0 1.0
80175 0 1.0
90 0 1.0
23562 0 0.8767286576584365
37292 0 0.9998879432674955
2617 0 1.0
38275 0 0.8563069650962937
367 0 1.0
3246 0 0.9999998807907033
5671 0 1.0
16 0 0.9999929666520943
92 0 0.9999916553499875
128009 0 0.9999890327454755


In [57]:
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct")

Loading checkpoint shards:  75%|███████▌  | 3/4 [00:04<00:01,  1.48s/it]

In [ ]:
model.cuda()

In [ ]:
prompt_tokens = tokenizer.apply_chat_template(
    row["input_messages"], 
    add_generation_prompt=True, 
    # chat_template=LLAMA_CARTRIDGE_TEMPLATE, 
    # add_special_tokens=False, 
    tokenize=True
)
prompt_and_answer_tokens = prompt_tokens + row["output_ids"]
num_answer_tokens = len(row["output_ids"])

In [ ]:
with torch.no_grad():
    input_ids = torch.tensor(prompt_and_answer_tokens).cuda().unsqueeze(0)
    outputs = model(input_ids)

In [ ]:
answer_logits = outputs.logits[:, -num_answer_tokens-1:-1]

In [ ]:
for answer_idx, (hf_token_logits, toka_topk_logprobs, toka_topk_ids) in enumerate(zip(answer_logits[0], row["topk_logprobs"], row["topk_token_ids"], strict=True)):
    hf_logprobs = F.softmax(hf_token_logits, dim=-1).log()
    hf_topk_logprobs, hf_topk_ids = torch.topk(hf_logprobs, k=10, dim=-1)
    hf_topk_ids = hf_topk_ids.tolist()
    hf_topk_logprobs = hf_topk_logprobs.tolist()
    print("HuggingFace Top 5 IDs and LogProbs:")
    for id, logprob in zip(hf_topk_ids[:5], hf_topk_logprobs[:5]):
        token = tokenizer.decode([id])
        print(f"  Token: {token!r:>12}, ID: {id:6d}, Prob: {math.exp(logprob):.4f}")
    
    print("\nToka Top 5 IDs and LogProbs:") 
    for id, logprob in zip(toka_topk_ids[:5], toka_topk_logprobs[:5]):
        token = tokenizer.decode([id])
        print(f"  Token: {token!r:>12}, ID: {id:6d}, Prob: {math.exp(logprob):.4f}")
    
    print("="*30)

HuggingFace Top 5 IDs and LogProbs:
  Token:         'To', ID:   1271, Prob: 0.9995
  Token:      'Based', ID:  29815, Prob: 0.0003
  Token:         '\\', ID:     59, Prob: 0.0001
  Token:        'The', ID:    791, Prob: 0.0000
  Token:          'I', ID:     40, Prob: 0.0000

Toka Top 5 IDs and LogProbs:
  Token:         'To', ID:   1271, Prob: 1.0000
  Token:        'The', ID:    791, Prob: 0.0000
  Token:         'to', ID:    998, Prob: 0.0000
  Token:        '_to', ID:   2401, Prob: 0.0000
  Token:        ' To', ID:   2057, Prob: 0.0000
HuggingFace Top 5 IDs and LogProbs:
  Token: ' determine', ID:   8417, Prob: 1.0000
  Token:  ' identify', ID:  10765, Prob: 0.0000
  Token:      ' find', ID:   1505, Prob: 0.0000
  Token:     ' solve', ID:  11886, Prob: 0.0000
  Token:    ' answer', ID:   4320, Prob: 0.0000

Toka Top 5 IDs and LogProbs:
  Token: ' determine', ID:   8417, Prob: 1.0000
  Token: ' determined', ID:  11075, Prob: 0.0000
  Token: ' determines', ID:  27667, Prob: 0.0000
  